## Fake NEws Classificer USsing Bidirectional LSTM

In [51]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd
import tensorflow as tf

In [52]:
df=pd.read_csv('/content/train.csv',engine='python',on_bad_lines='skip')

In [53]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [54]:
df.isnull().sum()

,0
id,0
title,565
author,1975
text,59
label,22


In [55]:
df.shape

(20823, 5)

In [56]:
df['label'] = pd.to_numeric(df['label'], errors='coerce')
df = df.dropna(subset=['label'])
df = df[df['label'].isin([0, 1])]
df['label'] = df['label'].astype(int)
# print("Shape of DataFrame after converting label to numeric, dropping nulls in label and filtering:", df.shape)

In [57]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [58]:
## get the independent features
X=df.drop('label',axis=1)

In [59]:
y=df['label']

In [60]:
## Check whether dataset is balanced or not
y.value_counts()

,count
label,
1,10411
0,10387


In [61]:
tf.__version__

'2.18.0'

In [62]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [63]:
## vocabulary size
voc_size=5000

In [64]:
messages=X.copy()

In [65]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...


In [66]:
import nltk
import re
from nltk.corpus import stopwords

In [67]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [68]:
messages.reset_index(inplace=True)

In [70]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # Handle potential non-string values in 'title'
    title = str(messages['title'][i]) if not pd.isna(messages['title'][i]) else ''
    review = re.sub('[^a-zA-Z]', ' ', title)
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [71]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [72]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1798, 2575, 4757, 4616, 4498, 96, 3931, 4032, 4042, 3906],
 [2019, 992, 4585, 1978, 646, 2989, 3024],
 [891, 256, 2734, 1013],
 [1954, 2340, 2061, 2936, 3239, 2693],
 [1292, 646, 3141, 2129, 812, 4197, 646, 448, 1125, 1384],
 [866,
  2670,
  4204,
  3037,
  3070,
  2936,
  2334,
  4120,
  4044,
  844,
  1670,
  3088,
  4793,
  519,
  3024],
 [3168, 3168, 1690, 290, 447, 255, 2942, 2520, 729, 3760, 3861, 3750],
 [3805, 4965, 3607, 4303, 3988, 1718, 4314, 2823, 882, 2086, 3416],
 [1293, 407, 1808, 200, 2936, 2497, 1941, 2133, 1239, 4953, 882, 2086, 3416],
 [1120, 1614, 955, 1323, 514, 1799, 2936, 2275, 882, 2086, 3416],
 [4412, 4872, 462, 3435, 4705, 2098, 3640, 4450, 2936, 3227],
 [3485, 2312, 1789, 3061, 1433, 2781, 219, 2285],
 [1378, 29, 1487, 1832, 4139, 4212, 2050, 1978, 2795, 3972, 4548],
 [2936, 2970, 4498, 2098, 2936, 514],
 [2757, 3646, 1284, 3524, 1980, 3695, 4690, 2196, 3911],
 [4224, 3245, 4320, 2641, 579, 2554, 3288, 882, 2086, 3416],
 [3096, 3490, 1442, 4161, 3716, 882, 

## Embedding Representation

In [73]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 4032, 4042, 3906],
       [   0,    0,    0, ...,  646, 2989, 3024],
       [   0,    0,    0, ...,  256, 2734, 1013],
       ...,
       [   0,    0,    0, ...,  882, 2086, 3416],
       [   0,    0,    0, ..., 2643, 1845, 3113],
       [   0,    0,    0, ...,  988, 3868,  468]], dtype=int32)

In [74]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [75]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [77]:
X_final.shape, y_final.shape

((20798, 20), (20798,))

In [78]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [79]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.8614 - loss: 0.3271 - val_accuracy: 0.9258 - val_loss: 0.1895
Epoch 2/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9532 - loss: 0.1216 - val_accuracy: 0.9298 - val_loss: 0.1881
Epoch 3/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9723 - loss: 0.0766 - val_accuracy: 0.9311 - val_loss: 0.2187
Epoch 4/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.9843 - loss: 0.0520 - val_accuracy: 0.9210 - val_loss: 0.2792
Epoch 5/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9944 - loss: 0.0222 - val_accuracy: 0.9263 - val_loss: 0.2830
Epoch 6/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9949 - loss: 0.0163 - val_accuracy: 0.9151 - val_loss: 0.3994
Epoch 7/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9969 - loss: 0.0095 - val_accuracy: 0.9186 - val_loss: 0.4169
Epoch 8/10
436/436 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9984 - loss: 0.0059 - val_accurac

In [80]:
y_pred=model.predict(X_test)

215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [81]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [82]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3159,  325],
       [ 236, 3144]])

In [83]:
print(accuracy_score(y_test,y_pred))

0.9182692307692307
